In [1]:
import geopandas as gpd
import numpy as np
import mapclassify as mc
import matplotlib.pyplot as plt

In [2]:
total_dict = {}
county_dict = {}
state_dict = {}

In [3]:
dynamic_dict = {}
dynamic_county_dict = {}
dynamic_state_dict = {}

1. Read County Data

In [4]:
counties_gdf = gpd.read_file('nyt_counties_data.geojson')

In [5]:
counties_gdf.head(5)

,NAME,state_name,population,fips,dt_first_case,dt_first_death,cases_ts,deaths_ts,today_case,today_new_case,today_death,today_new_death,dt_start,dt_end,dt_unit,geometry
0,Bladen,North Carolina,33778,37017,2020-03-29,None,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",4.0,2.0,0.0,0.0,2020-01-21,2020-04-21,day,"MULTIPOLYGON (((-78.90200 34.83527, -78.79960 ..."
1,Stanly,North Carolina,61114,37167,2020-03-20,2020-04-09,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",21.0,0.0,3.0,0.0,2020-01-21,2020-04-21,day,"MULTIPOLYGON (((-80.49738 35.20210, -80.29542 ..."
2,Summit,Ohio,541810,39153,2020-03-14,2020-03-27,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",385.0,15.0,28.0,4.0,2020-01-21,2020-04-21,day,"MULTIPOLYGON (((-81.68699 41.13596, -81.68495 ..."
3,Sullivan,Pennsylvania,6177,42113,2020-04-04,None,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",1.0,0.0,0.0,0.0,2020-01-21,2020-04-21,day,"MULTIPOLYGON (((-76.81373 41.59003, -76.22014 ..."
4,Upshur,Texas,40769,48459,2020-03-20,None,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",9.0,0.0,0.0,0.0,2020-01-21,2020-04-21,day,"MULTIPOLYGON (((-95.15274 32.66095, -95.15211 ..."


In [6]:
case = counties_gdf['today_case']
death = counties_gdf['today_death']
case_per_100k_capita = counties_gdf['today_case']/(counties_gdf['population']/100000)
death_per_100k_capita = counties_gdf['today_death']/(counties_gdf['population']/100000)
death_case_ratio = counties_gdf['today_death']/counties_gdf['today_case']
death_case_ratio = death_case_ratio.replace(np.nan, 0)

In [7]:
death_case_ratio

0       0.000000
1       0.142857
2       0.072727
3       0.000000
4       0.000000
5       0.076923
6       0.000000
7       0.000000
8       0.000000
9       0.000000
10      0.125000
11      0.000000
12      0.000000
13      0.000000
14      0.000000
15      0.000000
16      0.000000
17      0.000000
18      0.000000
19      0.000000
20      0.000000
21      0.040000
22      0.000000
23      0.000000
24      0.000000
25      0.000000
26      0.058824
27      0.000000
28      0.062500
29      0.000000
          ...   
3192    0.166667
3193    0.042204
3194    0.000000
3195    0.000000
3196    0.166667
3197    0.011765
3198    0.000000
3199    0.000000
3200    0.076923
3201    0.000000
3202    0.000000
3203    0.000000
3204    0.000000
3205    0.000000
3206    0.000000
3207    0.083333
3208    0.062500
3209    0.000000
3210    0.000000
3211    0.000000
3212    0.030000
3213    0.038627
3214    0.113924
3215    0.000000
3216    0.000000
3217    0.250000
3218    0.106667
3219    0.0598

1.1 Get Constant Classes from today's data

In [8]:
def create_dict(column,name,dict):
    tmp_dict = {}
    tmp_dict['Quantiles'] = {
        'bins': ','.join(mc.Quantiles(column, k=6).bins.astype(str)),
        'k': 6
    }
    tmp_dict['FisherJenks'] = {
        'bins': ','.join(mc.FisherJenks(column, k=6).bins.astype(str)),
        'k': 6
    }
    tmp_dict['NaturalBreaks'] = {
        'bins': ','.join(mc.NaturalBreaks(column, k=6).bins.astype(str)),
        'k': 6
    }
    dict[name] = tmp_dict

In [9]:
create_dict(case,'case',county_dict)

In [10]:
create_dict(death,'death',county_dict)

C:\Users\Jason\Anaconda3\lib\site-packages\mapclassify\classifiers.py:237: UserWarning: Warning: Not enough unique values in array to form k classes
  "Warning: Not enough unique values in array to form k classes", UserWarning
C:\Users\Jason\Anaconda3\lib\site-packages\mapclassify\classifiers.py:239: UserWarning: Warning: setting k to 4
  Warn("Warning: setting k to %d" % k_q, UserWarning)


In [11]:
create_dict(case_per_100k_capita,'case_per_100k_capita',county_dict)

In [12]:
create_dict(death_per_100k_capita,'death_per_100k_capita',county_dict)

C:\Users\Jason\Anaconda3\lib\site-packages\mapclassify\classifiers.py:237: UserWarning: Warning: Not enough unique values in array to form k classes
  "Warning: Not enough unique values in array to form k classes", UserWarning
C:\Users\Jason\Anaconda3\lib\site-packages\mapclassify\classifiers.py:239: UserWarning: Warning: setting k to 4
  Warn("Warning: setting k to %d" % k_q, UserWarning)


In [13]:
create_dict(death_case_ratio,'death_case_ratio',county_dict)

C:\Users\Jason\Anaconda3\lib\site-packages\mapclassify\classifiers.py:237: UserWarning: Warning: Not enough unique values in array to form k classes
  "Warning: Not enough unique values in array to form k classes", UserWarning
C:\Users\Jason\Anaconda3\lib\site-packages\mapclassify\classifiers.py:239: UserWarning: Warning: setting k to 4
  Warn("Warning: setting k to %d" % k_q, UserWarning)


In [14]:
county_dict

{'case': {'Quantiles': {'bins': '1.0,4.0,10.0,28.0,105.16666666666697,139335.0',
   'k': 6},
  'FisherJenks': {'bins': '909.0,3341.0,6576.0,15140.0,31079.0,139335.0',
   'k': 6},
  'NaturalBreaks': {'bins': '893.0,3154.0,6576.0,15140.0,31079.0,139335.0',
   'k': 6}},
 'death': {'Quantiles': {'bins': '0.0,1.0,4.0,10301.0', 'k': 6},
  'FisherJenks': {'bins': '45.0,206.0,544.0,1002.0,1717.0,10301.0', 'k': 6},
  'NaturalBreaks': {'bins': '46.0,206.0,544.0,1002.0,1717.0,10301.0', 'k': 6}},
 'case_per_100k_capita': {'Quantiles': {'bins': '4.5377016028157335,21.52112343882902,39.31514320454547,67.20708036142564,137.26098739617285,8703.12172637754',
   'k': 6},
  'FisherJenks': {'bins': '133.33708930768003,453.3259320936257,1166.7703795892967,2291.0087544708545,3262.7326150832514,8703.12172637754',
   'k': 6},
  'NaturalBreaks': {'bins': '133.33708930768003,453.3259320936257,1166.7703795892967,2291.0087544708545,3262.7326150832514,8703.12172637754',
   'k': 6}},
 'death_per_100k_capita': {'Qua

1.2 Create Dynamic classes

In [15]:
case_ts = counties_gdf['cases_ts'].apply(lambda x: x.split(','))
death_ts = counties_gdf['deaths_ts'].apply(lambda x: x.split(','))

In [16]:
length = len(case_ts[1])

In [17]:
new_case_dict = {}
for i in range(1,length):
    q6 = mc.Quantiles(case_ts.apply(lambda x: int(float(x[i]) - float(x[i - 1]))).values, k=6)
    new_case_dict[i] = ','.join(q6.bins.astype(int).astype(str))

C:\Users\Jason\Anaconda3\lib\site-packages\mapclassify\classifiers.py:237: UserWarning: Warning: Not enough unique values in array to form k classes
  "Warning: Not enough unique values in array to form k classes", UserWarning
C:\Users\Jason\Anaconda3\lib\site-packages\mapclassify\classifiers.py:239: UserWarning: Warning: setting k to 1
  Warn("Warning: setting k to %d" % k_q, UserWarning)
C:\Users\Jason\Anaconda3\lib\site-packages\mapclassify\classifiers.py:890: RuntimeWarning: invalid value encountered in double_scalars
  gadf = 1 - self.adcm / adam
C:\Users\Jason\Anaconda3\lib\site-packages\mapclassify\classifiers.py:239: UserWarning: Warning: setting k to 2
  Warn("Warning: setting k to %d" % k_q, UserWarning)
C:\Users\Jason\Anaconda3\lib\site-packages\mapclassify\classifiers.py:239: UserWarning: Warning: setting k to 3
  Warn("Warning: setting k to %d" % k_q, UserWarning)
C:\Users\Jason\Anaconda3\lib\site-packages\mapclassify\classifiers.py:239: UserWarning: Warning: setting k to 

In [18]:
new_death_dict = {}
for i in range(1,length):
    d6 = mc.Quantiles(death_ts.apply(lambda x: int(float(x[i]) - float(x[i - 1]))).values, k=6)
    new_death_dict[i] = ','.join(d6.bins.astype(int).astype(str))

In [19]:
dynamic_county_dict['new_case'] = new_case_dict
dynamic_county_dict['new_death'] = new_death_dict

2.Read State Data

Summary and ouput

In [22]:
import json
total_dict['county'] = county_dict
total_dict['state'] = state_dict
dynamic_dict['county'] = dynamic_county_dict
dynamic_county_dict['state'] = dynamic_state_dict
with open('classes.json','w') as json_file:
    json.dump(total_dict, json_file)
with open('dynamic_classes.json','w') as json_file:
    json.dump(dynamic_dict, json_file)
